In [1]:
from keras.preprocessing.image import ImageDataGenerator  # 引入用來擴增資料的類別
train_dir = "./train"  # 訓練資料放置的資料夾
# 再來是決定要對圖片做怎樣的變化，有旋轉、位移等各種參數可以調整
train_datagen = ImageDataGenerator(rescale=1/255,rotation_range=40,width_shift_range=0.2,\
                                   height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
# 最後是選擇從哪個資料夾拿圖片，還有決定圖片 size、batch_size 等
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150,150),batch_size=64,\
                                                    class_mode="categorical")

Using TensorFlow backend.


Found 2985 images belonging to 15 classes.


In [2]:
from keras.applications import Xception  #引入 Xception 的類別
from keras import models
from keras import layers
conv_base = Xception(weights="imagenet",include_top=False,input_shape=(150,150,3)) # 架 Xception
model = models.Sequential() # 建一個空的模型
model.add(conv_base) # 把前面架好的 Xception 放進模型
model.add(layers.Flatten()) # 把 Xception 抽取出的圖片特徵攤平
model.add(layers.Dense(256,activation="relu",kernel_initializer='random_normal')) # 再來就是通過幾層 Dense 來做分類
model.add(layers.Dropout(0.3)) # 加 Dropout 避免 overfitting
model.add(layers.Dense(128,activation="relu",kernel_initializer='random_normal'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(15,activation="softmax",kernel_initializer='random_normal'))
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["acc"])
model.fit_generator(train_generator,steps_per_epoch=100,epochs=10) # 用前面建好的 generator 做訓練

Epoch 1/10
100/100 [==============================] - 4317s 43s/step - loss: 1.5119 - acc: 0.5438
Epoch 2/10
100/100 [==============================] - 4241s 42s/step - loss: 0.8786 - acc: 0.7480
Epoch 3/10
100/100 [==============================] - 4236s 42s/step - loss: 0.6607 - acc: 0.8102
Epoch 4/10
100/100 [==============================] - 4220s 42s/step - loss: 0.6286 - acc: 0.8182
Epoch 5/10
100/100 [==============================] - 4237s 42s/step - loss: 0.5270 - acc: 0.8513
Epoch 6/10
100/100 [==============================] - 4236s 42s/step - loss: 0.4577 - acc: 0.8731
Epoch 7/10
100/100 [==============================] - 4238s 42s/step - loss: 0.3816 - acc: 0.8913
Epoch 8/10
100/100 [==============================] - 4230s 42s/step - loss: 0.3623 - acc: 0.9038
Epoch 9/10
100/100 [==============================] - 4233s 42s/step - loss: 0.3606 - acc: 0.9025
Epoch 10/10
100/100 [==============================] - 4238s 42s/step - loss: 0.3558 - acc: 0.9060


In [4]:
import pandas
import os
from keras.preprocessing import image # 引入讀取圖片用的類別
id_list = [] # 用來存圖片檔名
class_list = [] # 用來存分類結果
test_dir = "./dataset" # 測試資料放置的資料夾
submission_example = pandas.read_csv("img-submission.csv",header=0)
submission_example=submission_example["id"]
for i in submission_example:
  i=i+".jpg"
  file_name = os.path.join(test_dir,i)
  img = image.load_img(file_name,target_size=(150, 150)) # 讀取圖片並調整 size
  img_array = image.img_to_array(img) # 將圖片轉成陣列
  img_array = img_array/255 # 跟訓練資料一樣做 rescale
  img_array = img_array.reshape((1,)+img_array.shape) # reshape 成模型能吃的形狀
  class_predict = model.predict_classes(img_array) # 對圖片做類別預測
  id_list.append(i[:-4]) # 把圖片檔名後的 .jpg 拿掉
  class_list.append(class_predict[0])
submission = pandas.DataFrame({"id":id_list,"class":class_list}) # 建 DataFrame 存放結果
submission.to_csv("./result.csv",index = False, columns = ["id","class"]) # 輸出結果到 csv 檔